In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf

In [ ]:
# from kaggle_datasets import KaggleDatasets

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# import KaggleDatasets()

In [ ]:
# KaggleDatasets().get_gcs_path('tfrecsfinal')

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(user_credential)

In [ ]:
path1='gs://kds-fc9f65c439f9dc9618a2e866df97b928a4fbdc0f0dcf3542e537975a/'

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.data import Dataset, TFRecordDataset
from tensorflow.data.experimental import TFRecordWriter

In [ ]:
# train_df.columns

In [ ]:
# train_df=pd.read_csv('../input/ventilator-pressure-prediction/train.csv')

In [ ]:
# train_df.shape

In [ ]:
# train_df.head()

In [ ]:
# X=np.array(train_df[train_df.columns[2:len(train_df.columns)-1]]).astype('float32')

In [ ]:
# y=np.array(train_df['pressure']).astype('float32')

In [ ]:
# X1=X.reshape(6036000,5,1)

In [ ]:
# X1=tf.stack(X[0:1000000])

In [ ]:
# X2=tf.stack(X[1000000:2000000])

In [ ]:
# X3=tf.stack(X[2000000:3000000])

In [ ]:
# X4=tf.stack(X[3000000:4000000])

In [ ]:
# X5=tf.stack(X[5000000:6000000])

In [ ]:
# X6=tf.stack(X[6000000:])

In [ ]:
# print(X1.shape)
# print(X2.shape)
# print(X3.shape)
# print(X4.shape)
# print(X5.shape)
# print(X6.shape)

In [ ]:
# ds = Dataset.from_tensor_slices(X6)

In [ ]:
# ds_bytes = ds.map(tf.io.serialize_tensor)

In [ ]:
# import tensorflow.python_io

In [ ]:
# with tf.io.TFRecordWriter('tf1.tfrecord') as out_file:
#     out_file.write(ds_bytes)

In [ ]:
# writer = TFRecordWriter('./tfrec6.tfrecord')

In [ ]:
# writer.write(ds_bytes)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
pathlist=os.listdir('../input/tfrecsfinal')


In [ ]:
for x in range(len(pathlist)):
    pathlist[x]=path1+pathlist[x]

In [ ]:
pathlist

In [ ]:
ds_bytes_2 = TFRecordDataset(pathlist,num_parallel_reads=AUTO)
# ds_2 = ds_bytes_2.map(lambda x: tf.io.parse_tensor(x, out_type=tf.float64))

In [ ]:
# ds_bytes_2 = TFRecordDataset('gs://kds-2c1fe7c24a9609593d2503789706e6f8207fc1d2623f4900e0b28395/train1.tfrecord',num_parallel_reads=AUTO)
# ds_2 = ds_bytes_2.map(lambda x: tf.io.parse_tensor(x, out_type=tf.float64))

In [ ]:
# ds_bytes_2 = TFRecordDataset('../input/googtrain1/train1.tfrecord',num_parallel_reads=AUTO)

In [ ]:
ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = False

In [ ]:
ds_bytes_2 = ds_bytes_2.with_options(ignore_order)

In [ ]:
ds_3 = ds_bytes_2.map(decode)

In [ ]:
def decode(x):
    m=tf.io.parse_tensor(x, out_type=tf.float32)
    return (tf.reshape(m[1:6],(5,1)),tf.reshape(m[6],(1,)))

In [ ]:
# i=0
# for x in ds_3:
#     print(x)
#     i+=1
#     if i==1:
#         break

In [ ]:
# m2=tf.compat.v1.data.make_initializable_iterator(ds_2)

In [ ]:
# classifier.save_weights('./modelntrain.h5')

In [ ]:
# epoch=0
# for i in m1:
#     if(epoch>100 and epoch<200):
#         b1=tf.reshape(i[:,0:6],(128,6,1))
#         b2=i[:,6]
#         classifier.fit(b1,b2)
#         epoch+=1
#         print('epochfinish')
#     epoch+=1

In [ ]:
# classifier = Sequential()

# #Adding the input LSTM network layer
# classifier.add(LSTM(128, input_shape=(5,1), return_sequences=True))
# classifier.add(Dropout(0.2))
# #Adding a second LSTM network layer
# classifier.add(LSTM(128))

# #Adding a dense hidden layer
# classifier.add(Dense(64, activation='relu'))
# classifier.add(Dropout(0.2))

# #Adding the output layer
# classifier.add(Dense(1))
# classifier.compile(loss=tf.keras.losses.MeanSquaredError(),optimizer='adam')

In [ ]:
bb=ds_3.batch(1024)

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./tputftry1.h5',
    save_weights_only=True,
    monitor='loss',
    mode='min',
    verbose=1,save_best_only=True)

In [ ]:
with tpu_strategy.scope():
# #     load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
# #     classifier = tf.keras.models.load_model('../input/modeltry1', options=load_locally) 
    classifier = Sequential()

    #Adding the input LSTM network layer
    classifier.add(LSTM(128, input_shape=(5,1), return_sequences=True))
    classifier.add(Dropout(0.2))
    #Adding a second LSTM network layer
    classifier.add(LSTM(128))

    #Adding a dense hidden layer
    classifier.add(Dense(64, activation='relu'))
    classifier.add(Dropout(0.2))

    #Adding the output layer
    classifier.add(Dense(1))
    opt=tf.keras.optimizers.Adam(learning_rate=0.0001)
#     classifier.compile(loss=tf.keras.losses.MeanAbsoluteError(),optimizer=opt)
    classifier.compile(loss=tf.keras.losses.MeanAbsoluteError(),optimizer=opt,steps_per_execution=32)

In [ ]:
classifier.fit(bb,epochs=100,callbacks=[model_checkpoint_callback],steps_per_epoch=6036000//2048)